In [14]:
from selenium import webdriver
import pandas as pd

def validate_url(driver, url):
    ''' if valid url, return list of links otherwize return None'''
    try:
        driver.get(url)
        ids =  driver.find_elements_by_xpath('//*[@href]')
        return [x.get_attribute('href') for x in ids]
    except:
        return None

def bfs(seed_url, driver, frames, limit):
    ''' get all links from a given url'''
    
    depth = 0
    node_id = 0
    parent = 0
    
    frames.append(pd.DataFrame(dict(depth=[depth], node_id=[node_id], parent=[parent], url=[seed_url])))
    depth += 1
    
    links = validate_url(driver, seed_url)
    new_node_id = len(links) + node_id + 1
    temp_frame = pd.DataFrame(dict(node_id=list(range(node_id + 1,new_node_id)), url=links))
    temp_frame['depth'] = depth
    temp_frame['parent'] = parent
    frames.append(temp_frame)
    return pd.concat(frames)
#     for i in range(depth, limit):
#         for link in ids:
#             driver.get(link)
#             ids = driver.find_elements_by_xpath('//*[@href]')
#             limit.append()

In [15]:
driver=webdriver.Firefox()
frames = list()
tr = bfs("https://portland.craigslist.org/", driver, frames, 1)

driver.close()
tr

,depth,node_id,parent,url
0,0,0,0,https://portland.craigslist.org/
0,1,1,0,https://portland.craigslist.org/
1,1,2,0,https://www.craigslist.org/styles/cl.css?v=976...
2,1,3,0,https://www.craigslist.org/styles/homepage.css...
3,1,4,0,https://www.craigslist.org/styles/jquery-ui-cl...
4,1,5,0,https://www.craigslist.org/js/search-concat.mi...
5,1,6,0,https://portland.craigslist.org/
6,1,7,0,https://portland.craigslist.org/
7,1,8,0,https://post.craigslist.org/c/pdx
8,1,9,0,https://accounts.craigslist.org/login/home
